## Let's make prediction.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import joblib


In [2]:
df=pd.read_csv('./map_data/map.csv')

In [3]:
df.shape

(32760, 25)

In [4]:
df.sample

<bound method NDFrame.sample of          Dist_Fault  Dist_River    Dist_Road  Geology_Nepal_ID  \
0       6189.781845  311.401100   575.556999                28   
1       6438.144934  139.358780   798.184286                28   
2       6438.144934  139.358780   798.184286                28   
3       6686.508132  109.252238  1020.811671                28   
4       6686.508132  109.252238  1020.811671                28   
...             ...         ...          ...               ...   
32755  17274.471301  301.537674   213.826582                14   
32756  17340.859708  466.789022   150.952697                14   
32757  17340.859708  466.789022   150.952697                14   
32758  17410.545547  510.865814   149.423242                14   
32759  17410.545547  510.865814   149.423242                14   

       Geology_geo8apg_ID  LULC         NDVI      aspect        date  \
0                       0    80  2247.130435    0.000000  2024-01-01   
1                       0    80

In [5]:
df_calc=df.copy()

In [6]:
# Calculate TPI (Topographic Position Index)
# Since we don't have neighborhood elevation, we approximate using a rolling window mean
window_size = 5  # Adjust based on dataset resolution
df_calc["TPI"] = df_calc["elevation"] - df_calc["elevation"].rolling(window=window_size, center=True).mean()

# Calculate TRI (Terrain Roughness Index) as standard deviation of elevation in a window
df_calc["TRI"] = df_calc["elevation"].rolling(window=window_size, center=True).std()

# Calculate SPI (Stream Power Index) = A * tan(slope)
# Here, we don't have upslope contributing area (A), so we approximate with elevation
df_calc["SPI"] = df_calc["elevation"] * np.tan(np.radians(df_calc["slope"]))

# Calculate TWI (Topographic Wetness Index) = ln(A / tan(slope))
# Again, approximating A with elevation
df_calc["TWI"] = np.log(df_calc["elevation"] / np.tan(np.radians(df_calc["slope"])))

# Display the first few rows with new indices
df_calc[["elevation", "slope", "TPI", "TRI", "SPI", "TWI"]].head()


,elevation,slope,TPI,TRI,SPI,TWI
0,180,0.927410,NaN,NaN,2.913799,9.316456
1,179,0.000000,NaN,NaN,0.000000,inf
2,179,0.927410,-3.6,5.504544,2.897611,9.310885
3,183,3.704792,-2.4,7.162402,11.849448,7.946691
4,192,2.129314,3.2,7.395945,7.138686,8.549462


In [7]:
df=pd.merge(df,df_calc[["TPI", "TRI", "SPI", "TWI"]],left_index=True,right_index=True)
df.shape

(32760, 29)

In [8]:
categorical_columns=['LULC','Geology_Nepal_ID']
numerical_columns=['Bulk_Density',
 'CEC',
 'Clay_Content',
 'Coarse_Fragments',
 'Dist_Fault',
 'Dist_River',
 'Dist_Road',
 'NDVI',
 'Nitrogen',
 'Sand_Content',
 'Silt_Content',
 'Soil_Organic_Carbon',
 'Soil_pH',
 'aspect',
 'elevation',
 'precipitation',
 'rainfall_360d',
 'rainfall_7d',
 'slope',
 'TPI',
 'TRI',
 'SPI',
 'TWI']
target_column=['landslide']

In [9]:
GEOLOGY_NEPAL_ID = df['Geology_Nepal_ID'].unique()

In [10]:
def clean_data_inconsistencies(df):
    df=df[df['TWI']<=9999]
    df.dropna(inplace=True)
    df['Geology_Nepal_ID'] = df['Geology_Nepal_ID'].replace(GEOLOGY_NEPAL_ID, range(len(GEOLOGY_NEPAL_ID)))
    return df

In [11]:
preprocessor= joblib.load('preprocessor.pkl')

In [12]:
df=clean_data_inconsistencies(df)
preprocessed_data=preprocessor.transform(df)

/var/folders/1f/rff5j6dx6wqbh_t1yts0t1nm0000gn/T/ipykernel_1856/985869703.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/var/folders/1f/rff5j6dx6wqbh_t1yts0t1nm0000gn/T/ipykernel_1856/985869703.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Geology_Nepal_ID'] = df['Geology_Nepal_ID'].replace(GEOLOGY_NEPAL_ID, range(len(GEOLOGY_NEPAL_ID)))


In [13]:
xg_boost_model=joblib.load('best_xgboost.pkl')

In [14]:
import numpy as np
from scipy.sparse import csr_matrix

# Convert to sparse matrix if necessary
X_test = csr_matrix(preprocessed_data)

print("Preprocessing complete.")

Preprocessing complete.


In [15]:
# Load the best trained model (change filename if needed)
best_model = joblib.load("best_random_forest.pkl")  # or "best_xgboost.pkl"

# Predict landslide probability
predictions = best_model.predict_proba(X_test)[:, 1]  # Extract probability of landslide

# Add predictions to dataframe
df["landslide_probability"] = predictions

print("Predictions completed.")


Predictions completed.


In [16]:
df.sample(10)

,Dist_Fault,Dist_River,Dist_Road,Geology_Nepal_ID,Geology_geo8apg_ID,LULC,NDVI,aspect,date,elevation,...,Coarse_Fragments,Nitrogen,Sand_Content,Silt_Content,Soil_pH,TPI,TRI,SPI,TWI,landslide_probability
12123,6242.293988,1821.360982,1859.725512,5,2482,10,6247.869565,190.967675,2024-01-01,976,...,165.0,2597.0,309.0,429.0,56.0,-23.0,96.470203,563.262415,7.433180,0.480074
23816,12248.320439,252.626880,574.342509,5,2482,10,6227.434783,276.661054,2024-01-01,660,...,160.0,3418.0,297.0,492.0,56.0,-4.6,50.594466,644.744242,6.515626,0.557847
3491,9835.555261,1430.728914,897.761130,1,0,10,6962.086957,168.885056,2024-01-01,300,...,151.0,2123.0,366.0,393.0,58.0,-5.6,58.722227,113.830808,6.672852,0.330178
2410,10086.138805,1078.805574,983.188515,2,0,10,6550.913043,308.822434,2024-01-01,412,...,123.0,2521.0,336.0,423.0,58.0,-4.2,31.610125,106.384857,7.374983,0.411667
1145,8945.865500,1026.055644,709.489805,2,0,10,6968.260870,171.432874,2024-01-01,326,...,131.0,2305.0,335.0,429.0,58.0,10.4,9.071935,80.051411,7.191126,0.444728
4031,9070.545728,1183.912032,879.702733,1,0,10,4661.565217,131.517491,2024-01-01,212,...,116.0,1614.0,415.0,369.0,61.0,0.0,4.582576,10.354724,8.375730,0.244487
6838,4412.002910,82.939311,579.647412,6,2482,10,4144.260870,265.402043,2024-01-01,290,...,153.0,5830.0,355.0,384.0,57.0,9.0,26.153394,351.366402,5.477932,0.714942
25940,13874.693389,1332.805029,478.404476,5,2482,10,5911.391304,308.364449,2024-01-01,1127,...,184.0,3081.0,312.0,467.0,56.0,-1.4,28.823601,499.695170,7.840631,0.735299
24540,12384.876396,373.673515,247.227291,5,2482,10,6687.000000,178.094952,2024-01-01,487,...,155.0,2816.0,268.0,512.0,56.0,-30.6,92.362330,268.185335,6.784850,0.595776
29018,16144.653118,490.319216,1159.083685,5,2482,10,6787.782609,334.445718,2024-01-01,729,...,164.0,3412.0,306.0,471.0,55.0,20.2,63.766763,680.183601,6.660985,0.658064


In [17]:
df.columns

Index(['Dist_Fault', 'Dist_River', 'Dist_Road', 'Geology_Nepal_ID',
       'Geology_geo8apg_ID', 'LULC', 'NDVI', 'aspect', 'date', 'elevation',
       'precipitation', 'rainfall_360d', 'rainfall_7d', 'slope', 'longitude',
       'latitude', 'Bulk_Density', 'Soil_Organic_Carbon', 'CEC',
       'Clay_Content', 'Coarse_Fragments', 'Nitrogen', 'Sand_Content',
       'Silt_Content', 'Soil_pH', 'TPI', 'TRI', 'SPI', 'TWI',
       'landslide_probability'],
      dtype='object')

In [18]:
df.to_csv('landslide_probability.csv',index=False)

In [ ]:

import geopandas as gpd
import folium
from folium.plugins import MarkerCluster

# Load landslide susceptibility data
landslide_df = pd.read_csv('landslide_probability.csv')
past_records_df = pd.read_csv('./map_data/final_landslide_features.csv')

# Define study area polygon
boundary = ee.Geometry.Polygon([
    [[84.39506358183932, 27.86930215221273],
     [84.39506358183932, 27.688263663637958],
     [84.57427805937839, 27.688263663637958],
     [84.57427805937839, 27.86930215221273]]
])

# Convert landslide probability data into an Image
landslide_points = [ee.Feature(ee.Geometry.Point([row['longitude'], row['latitude']]),
                               {'probability': row['landslide_probability']})
                    for _, row in landslide_df.iterrows()]

landsusceptibility = ee.FeatureCollection(landslide_points).reduceToImage(
    properties=['probability'], reducer=ee.Reducer.mean()
)

# Define visualization parameters
vis_params = {
    'min': landslide_df['landslide_probability'].min(),
    'max': landslide_df['landslide_probability'].max(),
    'palette': ['green', 'yellow', 'orange', 'red']
}

# Create a GEE map
Map = geemap.Map()
Map.centerObject(boundary, 12)
Map.addLayer(landsusceptibility, vis_params, 'Landslide Susceptibility')

# Create a Folium FeatureGroup to store markers
marker_cluster = folium.FeatureGroup(name="Past Landslide Events")

# Add past landslide records as points
for _, row in past_records_df.iterrows():
    marker = folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Landslide Event: {row['date']}",
        icon=folium.Icon(color='black', icon='info-sign')
    )
    marker_cluster.add_child(marker)

# Add the markers to the geemap Map
Map.add_layer(marker_cluster)

# Display the map
Map
# Add a legend
# Convert color names to RGB tuples
legend_dict = {
    'Low Susceptibility': (0, 255, 0),     # Green
    'Moderate Susceptibility': (255, 255, 0),  # Yellow
    'High Susceptibility': (255, 165, 0),  # Orange
    'Very High Susceptibility': (255, 0, 0)  # Red
}

# Add the legend
Map.add_legend(title="Landslide Susceptibility Scale", legend_dict=legend_dict)

# Display the map
Map

Map(center=[27.778786690505562, 84.48467082060839], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
import geopandas as gpd
import pandas as pd
import ee
import geemap
import folium
from folium.plugins import MarkerCluster

# Initialize Earth Engine
ee.Initialize()

# Load landslide susceptibility data
landslide_df = pd.read_csv('landslide_probability.csv')
past_records_df = pd.read_csv('./map_data/final_landslide_features.csv')

# Define study area polygon
boundary = ee.Geometry.Polygon([
    [[84.39506358183932, 27.86930215221273],
     [84.39506358183932, 27.688263663637958],
     [84.57427805937839, 27.688263663637958],
     [84.57427805937839, 27.86930215221273]]
])

# Convert landslide probability data into points
landslide_points = [ee.Feature(ee.Geometry.Point([row['longitude'], row['latitude']]),
                              {'probability': float(row['landslide_probability'])})
                   for _, row in landslide_df.iterrows()]

# Create a feature collection from the points
fc = ee.FeatureCollection(landslide_points)

# Interpolate points to create a continuous surface
# Using inverse distance weighted interpolation
landslide_susceptibility = fc.kriging(
    propertyName='probability',
    shape='exponential',
    range=0.05,  # Adjust based on your data density
    sill=1.0,
    nugget=0.1,
    maxDistance=0.1,  # Maximum distance to consider for interpolation
    reducer=ee.Reducer.mean(),
    outputName='probability'
).clip(boundary)

# Alternative approach using kernel density estimation
# kernel_radius = 0.01  # Adjust based on desired smoothness
# landslide_susceptibility = fc.reduceToImage(
#     properties=['probability'],
#     reducer=ee.Reducer.mean()
# ).focal_mean(kernel_radius, 'circle', 'pixels')

# Define visualization parameters
vis_params = {
    'min': 0,
    'max': 1,  # Assuming probability ranges from 0 to 1
    'palette': ['green', 'yellow', 'orange', 'red']
}

# Create a GEE map
Map = geemap.Map()
Map.centerObject(boundary, 12)
Map.addLayer(landslide_susceptibility, vis_params, 'Landslide Susceptibility')

# Add OSM as a base layer
Map.add_basemap('OpenStreetMap')

# Create a Folium FeatureGroup to store markers
marker_cluster = folium.plugins.MarkerCluster(name="Past Landslide Events")

# Add past landslide records as points
for _, row in past_records_df.iterrows():
    popup_html = f"""
    <b>Landslide Event</b><br>
    Date: {row['date']}<br>
    """
    if 'magnitude' in row:
        popup_html += f"Magnitude: {row['magnitude']}<br>"
    
    marker = folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color='black', icon='info-sign')
    )
    marker_cluster.add_child(marker)

# Add the markers to the map
Map.add_child(marker_cluster)

# Add a legend
legend_dict = {
    'Low Susceptibility': (0, 255, 0),  # Green
    'Moderate Susceptibility': (255, 255, 0),  # Yellow
    'High Susceptibility': (255, 165, 0),  # Orange
    'Very High Susceptibility': (255, 0, 0)  # Red
}

# Add the legend
Map.add_legend(title="Landslide Susceptibility Scale", legend_dict=legend_dict)

# Add scale and other map controls
Map.add_scale_bar()
Map.add_search_control()
Map.add_layer_control()

# Display the map
Map

TypeError: FeatureCollection.kriging() got an unexpected keyword argument 'outputName'

In [29]:
import geopandas as gpd
import pandas as pd
import ee
import geemap
import folium
from folium.plugins import MarkerCluster

# Initialize Earth Engine
ee.Initialize()

# Load landslide susceptibility data
landslide_df = pd.read_csv('landslide_probability.csv')
past_records_df = pd.read_csv('./map_data/final_landslide_features.csv')

# Define study area polygon
boundary = ee.Geometry.Polygon([
    [[84.39506358183932, 27.86930215221273],
     [84.39506358183932, 27.688263663637958],
     [84.57427805937839, 27.688263663637958],
     [84.57427805937839, 27.86930215221273]]
])

# Convert landslide probability data into points
landslide_points = [ee.Feature(ee.Geometry.Point([row['longitude'], row['latitude']]),
                              {'probability': float(row['landslide_probability'])})
                   for _, row in landslide_df.iterrows()]

# Create a feature collection from the points
fc = ee.FeatureCollection(landslide_points)

# APPROACH 1: Using focal_mean for smoothing
# First convert points to an image
landslide_image = fc.reduceToImage(
    properties=['probability'],
    reducer=ee.Reducer.mean()
)

# Apply smoothing with a focal mean filter
kernel_radius = 500  # in meters - adjust based on desired smoothness
landslide_susceptibility = landslide_image.focal_mean(
    radius=kernel_radius, 
    kernelType='circle', 
    units='meters'
).clip(boundary)

# APPROACH 2: Alternative using idw interpolation
# Uncomment this section if you prefer IDW interpolation
# from sklearn.neighbors import KNeighborsRegressor
# 
# # Extract points for interpolation
# points = landslide_df[['longitude', 'latitude']].values
# values = landslide_df['landslide_probability'].values
# 
# # Create a grid of points for prediction
# x_min, y_min = boundary.bounds()[0:2].getInfo()
# x_max, y_max = boundary.bounds()[2:4].getInfo()
# grid_size = 0.001  # Adjust for resolution
# x_grid = np.arange(x_min, x_max, grid_size)
# y_grid = np.arange(y_min, y_max, grid_size)
# xx, yy = np.meshgrid(x_grid, y_grid)
# grid_points = np.vstack([xx.ravel(), yy.ravel()]).T
# 
# # Perform IDW interpolation
# idw_model = KNeighborsRegressor(n_neighbors=12, weights='distance')
# idw_model.fit(points, values)
# grid_values = idw_model.predict(grid_points)
# 
# # Create new dataframe with interpolated values
# interpolated_df = pd.DataFrame({
#     'longitude': grid_points[:, 0],
#     'latitude': grid_points[:, 1],
#     'landslide_probability': grid_values
# })
# 
# # Convert to Earth Engine
# interp_points = [ee.Feature(ee.Geometry.Point([row['longitude'], row['latitude']]),
#                           {'probability': float(row['landslide_probability'])})
#                for _, row in interpolated_df.iterrows()]
# interp_fc = ee.FeatureCollection(interp_points)
# landslide_susceptibility = interp_fc.reduceToImage(
#     properties=['probability'],
#     reducer=ee.Reducer.mean()
# ).clip(boundary)

# Define visualization parameters
vis_params = {
    'min': landslide_df['landslide_probability'].min(),
    'max': landslide_df['landslide_probability'].max(),
    'palette': ['green', 'yellow', 'orange', 'red']
}

# Create a GEE map
Map = geemap.Map()
Map.centerObject(boundary, 12)
Map.addLayer(landslide_susceptibility, vis_params, 'Landslide Susceptibility')

# Add OSM as a base layer
Map.add_basemap('OpenStreetMap')

# Create a Folium FeatureGroup to store markers
marker_cluster = MarkerCluster(name="Past Landslide Events")

# Add past landslide records as points
for _, row in past_records_df.iterrows():
    popup_html = f"<b>Landslide Event</b><br>Date: {row['date']}"
    if 'magnitude' in row:
        popup_html += f"<br>Magnitude: {row['magnitude']}"
    
    marker = folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color='black', icon='info-sign')
    )
    marker_cluster.add_child(marker)

# Add the markers to the map
Map.add_layer(marker_cluster)

# Add a legend
legend_dict = {
    'Low Susceptibility': (0, 255, 0),  # Green
    'Moderate Susceptibility': (255, 255, 0),  # Yellow
    'High Susceptibility': (255, 165, 0),  # Orange
    'Very High Susceptibility': (255, 0, 0)  # Red
}

# Add the legend
Map.add_legend(title="Landslide Susceptibility Scale", legend_dict=legend_dict)

# Display the map
Map

Map(center=[27.778786690505562, 84.48467082060839], controls=(WidgetControl(options=['position', 'transparent_…

In [33]:
import geopandas as gpd
import pandas as pd
import ee
import geemap
import folium
from folium.plugins import MarkerCluster



# Load landslide susceptibility data
landslide_df = pd.read_csv('landslide_probability.csv')
past_records_df = pd.read_csv('./map_data/final_landslide_features.csv')

# Define study area polygon
boundary = ee.Geometry.Polygon([
    [[84.39506358183932, 27.86930215221273],
     [84.39506358183932, 27.688263663637958],
     [84.57427805937839, 27.688263663637958],
     [84.57427805937839, 27.86930215221273]]
])

# Convert landslide probability data into points
landslide_points = [ee.Feature(ee.Geometry.Point([row['longitude'], row['latitude']]),
                              {'probability': float(row['landslide_probability'])})
                   for _, row in landslide_df.iterrows()]

# Create a feature collection from the points
fc = ee.FeatureCollection(landslide_points)

# First convert points to an image
landslide_image = fc.reduceToImage(
    properties=['probability'],
    reducer=ee.Reducer.mean()
)

# Apply smoothing with a larger focal mean filter for better continuity
kernel_radius = 800  # in meters - increased for smoother effect
landslide_susceptibility = landslide_image.focal_mean(
    radius=kernel_radius, 
    kernelType='circle', 
    units='meters'
).clip(boundary)

# Get a Digital Elevation Model (DEM) for topographic context
dem = ee.Image('USGS/SRTMGL1_003').clip(boundary)
hillshade = ee.Terrain.hillshade(dem).clip(boundary)

# Define visualization parameters with semi-transparency
vis_params = {
    'min': landslide_df['landslide_probability'].min(),
    'max': landslide_df['landslide_probability'].max(),
    'palette': ['green', 'yellow', 'orange', 'red'],
    'opacity': 0.7  # Add transparency to blend with topography
}

# Create a GEE map
Map = geemap.Map()
Map.centerObject(boundary, 12)

# Add hillshade as a base layer for topographic context
Map.addLayer(hillshade, {'min': 0, 'max': 255}, 'Hillshade', True, 0.5)

# Add a terrain base map first
Map.add_basemap('TERRAIN')

# Add the landslide susceptibility layer with transparency
Map.addLayer(landslide_susceptibility, vis_params, 'Landslide Susceptibility')

# Create a Folium FeatureGroup to store markers
marker_cluster = MarkerCluster(name="Past Landslide Events")

# Add past landslide records as points
for _, row in past_records_df.iterrows():
    popup_html = f"<b>Landslide Event</b><br>Date: {row['date']}"
    if 'magnitude' in row:
        popup_html += f"<br>Magnitude: {row['magnitude']}"
    
    marker = folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color='black', icon='info-sign')
    )
    marker_cluster.add_child(marker)

# Add the markers to the map
Map.add_layer(marker_cluster)

# Add a legend
legend_dict = {
    'Low Susceptibility': (0, 255, 0),  # Green
    'Moderate Susceptibility': (255, 255, 0),  # Yellow
    'High Susceptibility': (255, 165, 0),  # Orange
    'Very High Susceptibility': (255, 0, 0)  # Red
}

# Add the legend
Map.add_legend(title="Landslide Susceptibility Scale", legend_dict=legend_dict)

# Add layer control
Map.add_layer_control()

# Display the map
Map

Map(center=[27.778786690505562, 84.48467082060839], controls=(WidgetControl(options=['position', 'transparent_…

In [21]:
import geopandas as gpd
import pandas as pd
import numpy as np
import ee
import geemap
import folium
from folium.plugins import MarkerCluster
import os
from PIL import Image
from IPython.display import display

# Load landslide susceptibility data
landslide_df = pd.read_csv('landslide_probability.csv')
past_records_df = pd.read_csv('./map_data/final_landslide_features.csv')

# Define study area polygon
boundary = ee.Geometry.Polygon([
    [[84.39506358183932, 27.86930215221273],
     [84.39506358183932, 27.688263663637958],
     [84.57427805937839, 27.688263663637958],
     [84.57427805937839, 27.86930215221273]]
])

# Convert landslide probability data into points
landslide_points = [ee.Feature(ee.Geometry.Point([row['longitude'], row['latitude']]),
                              {'probability': float(row['landslide_probability'])})
                   for _, row in landslide_df.iterrows()]

# Create a feature collection from the points
fc = ee.FeatureCollection(landslide_points)

# Convert points to an image
landslide_image = fc.reduceToImage(
    properties=['probability'],
    reducer=ee.Reducer.mean()
)

# Apply smoothing with a focal mean filter
kernel_radius = 1000  # in meters - increased for even smoother effect
landslide_susceptibility = landslide_image.focal_mean(
    radius=kernel_radius, 
    kernelType='circle', 
    units='meters'
).clip(boundary)

# Get DEM for topographic context
dem = ee.Image('USGS/SRTMGL1_003').clip(boundary)
hillshade = ee.Terrain.hillshade(dem, 315, 45).clip(boundary)
slope = ee.Terrain.slope(dem).clip(boundary)

# Generate contour lines
contours = dem.convolve(ee.Kernel.gaussian(5)).subtract(0).divide(20).toInt()
contours = contours.mask(contours.subtract(contours.convolve(ee.Kernel.fixed(3, 3, [[1, 1, 1], [1, 0, 1], [1, 1, 1]]))))

# Create a blended landslide susceptibility and terrain visualization
# Normalize DEM for the study area for better visualization
min_elev = dem.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=boundary,
    scale=30,
    maxPixels=1e9
).get('elevation')

max_elev = dem.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=boundary,
    scale=30,
    maxPixels=1e9
).get('elevation')

dem_normalized = dem.subtract(min_elev).divide(ee.Number(max_elev).subtract(min_elev))

# Define visualization parameters with semi-transparency
susceptibility_vis = {
    'min': landslide_df['landslide_probability'].min(),
    'max': landslide_df['landslide_probability'].max(),
    'palette': ['#00ff00', '#ffff00', '#ff9900', '#ff0000'],  # Brighter colors for better visibility
    'opacity': 0.7
}

hillshade_vis = {
    'min': 0,
    'max': 255,
    'palette': ['#ffffff', '#f0f0f0', '#d0d0d0', '#a0a0a0', '#808080'],
}

# Create a GEE map
Map = geemap.Map()
Map.centerObject(boundary, 12)

# Add basemaps
Map.add_basemap('HYBRID', shown=False)  # Satellite with labels
Map.add_basemap('TERRAIN')  # Terrain base

# Add hillshade for topographic context
Map.addLayer(hillshade, hillshade_vis, 'Hillshade', True, 0.8)

# Add contour lines
Map.addLayer(contours, {'palette': '#000000'}, 'Contour Lines', True, 0.4)

# Add World Topo Map from ESRI
Map.add_basemap('Esri.WorldTopoMap', shown=True, opacity=0.6)
Map.add_basemap('OpenStreetMap', shown=False)  # OpenStreetMap

# Add the landslide susceptibility layer with transparency
Map.addLayer(landslide_susceptibility, susceptibility_vis, 'Landslide Susceptibility')

# Filter past landslide records to ensure they are within the boundary
# Convert boundary to a list of coordinates for filtering
boundary_coords = boundary.coordinates().getInfo()[0]
min_lon = min(point[0] for point in boundary_coords)
max_lon = max(point[0] for point in boundary_coords)
min_lat = min(point[1] for point in boundary_coords)
max_lat = max(point[1] for point in boundary_coords)

# Filter past_records_df to include only points within the boundary
filtered_records = past_records_df[
    (past_records_df['longitude'] >= min_lon) & 
    (past_records_df['longitude'] <= max_lon) & 
    (past_records_df['latitude'] >= min_lat) & 
    (past_records_df['latitude'] <= max_lat)
]

# Create a Folium FeatureGroup to store markers
marker_cluster = MarkerCluster(name="Past Landslide Events")

# Add past landslide records as points with better visibility
for _, row in filtered_records.iterrows():
    popup_html = f"""
    <div style="min-width:200px">
        <h4>Landslide Event</h4>
        <b>Date:</b> {row['date']}<br>
    """
    if 'magnitude' in row:
        popup_html += f"<b>Magnitude:</b> {row['magnitude']}<br>"
    if 'area' in row:
        popup_html += f"<b>Area:</b> {row['area']} m²<br>"
    popup_html += "</div>"
    
    marker = folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color='red', icon='warning-sign')  # Changed to more visible icon
    )
    marker_cluster.add_child(marker)

# Add the markers to the map
Map.add_layer(marker_cluster)

# Add a legend
legend_dict = {
    'Low Susceptibility': (0, 255, 0),      # Green
    'Moderate Susceptibility': (255, 255, 0),  # Yellow
    'High Susceptibility': (255, 165, 0),    # Orange
    'Very High Susceptibility': (255, 0, 0)   # Red
}

# Add the legend
Map.add_legend(title="Landslide Susceptibility Scale", legend_dict=legend_dict)

# Add map controls
Map.add_layer_control()


# Function to export map as high-resolution PNG
def export_map_as_png(m, output_path='landslide_susceptibility_map.png', width=3000, height=2000):
    """Export the map as a high-resolution PNG file"""
    m.to_html('temp_map.html')
    
    # Use selenium to render the map at high resolution
    try:
        from selenium import webdriver
        from selenium.webdriver.chrome.options import Options
        from selenium.webdriver.chrome.service import Service
        from webdriver_manager.chrome import ChromeDriverManager
        
        options = Options()
        options.add_argument('--headless')
        options.add_argument(f'--window-size={width},{height}')
        
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get('file://' + os.path.abspath('temp_map.html'))
        
        # Wait for map to load
        import time
        time.sleep(5)
        
        # Take screenshot
        driver.save_screenshot(output_path)
        driver.quit()
        
        print(f"Map exported as {output_path}")
        return True
    except Exception as e:
        print(f"Error exporting map: {e}")
        print("Falling back to regular screenshot method...")
        
        # Alternative: Use screenshot method (lower resolution)
        m.save('temp_map.html')
        print(f"Map saved as HTML. Please open in a browser and take a screenshot manually.")
        return False

# Add button to export high-resolution map
export_button = folium.LayerControl(position='topright')
Map.add_layer(export_button)

# Display the map
Map

# Export high-resolution map
export_map_as_png(Map, 'high_resolution_landslide_map.png')

print("To save a high-resolution version of this map, use the export_map_as_png() function.")

Error exporting map: No module named 'webdriver_manager'
Falling back to regular screenshot method...
Map saved as HTML. Please open in a browser and take a screenshot manually.
To save a high-resolution version of this map, use the export_map_as_png() function.


In [42]:
import geopandas as gpd
import pandas as pd
import numpy as np
import ee
import geemap
import folium
from folium.plugins import MarkerCluster
import os
from PIL import Image
from IPython.display import display

# Load landslide susceptibility data
landslide_df = pd.read_csv('landslide_probability.csv')
past_records_df = pd.read_csv('./map_data/final_landslide_features.csv')

# Define study area polygon
boundary = ee.Geometry.Polygon([
    [[84.39506358183932, 27.86930215221273],
     [84.39506358183932, 27.688263663637958],
     [84.57427805937839, 27.688263663637958],
     [84.57427805937839, 27.86930215221273]]
])

# Convert landslide probability data into points
landslide_points = [ee.Feature(ee.Geometry.Point([row['longitude'], row['latitude']]),
                              {'probability': float(row['landslide_probability'])})
                   for _, row in landslide_df.iterrows()]

# Create a feature collection from the points
fc = ee.FeatureCollection(landslide_points)

# Convert points to an image
landslide_image = fc.reduceToImage(
    properties=['probability'],
    reducer=ee.Reducer.mean()
)

# Apply smoothing with a focal mean filter
kernel_radius = 1000  # in meters - increased for even smoother effect
landslide_susceptibility = landslide_image.focal_mean(
    radius=kernel_radius, 
    kernelType='circle', 
    units='meters'
).clip(boundary)

# Get DEM for topographic context
dem = ee.Image('USGS/SRTMGL1_003').clip(boundary)
hillshade = ee.Terrain.hillshade(dem, 315, 45).clip(boundary)
slope = ee.Terrain.slope(dem).clip(boundary)

# Generate contour lines
contours = dem.convolve(ee.Kernel.gaussian(5)).subtract(0).divide(20).toInt()
contours = contours.mask(contours.subtract(contours.convolve(ee.Kernel.fixed(3, 3, [[1, 1, 1], [1, 0, 1], [1, 1, 1]]))))

# Create a blended landslide susceptibility and terrain visualization
# Normalize DEM for the study area for better visualization
min_elev = dem.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=boundary,
    scale=30,
    maxPixels=1e9
).get('elevation')

max_elev = dem.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=boundary,
    scale=30,
    maxPixels=1e9
).get('elevation')

dem_normalized = dem.subtract(min_elev).divide(ee.Number(max_elev).subtract(min_elev))

# Define visualization parameters with semi-transparency
susceptibility_vis = {
    'min': landslide_df['landslide_probability'].min(),
    'max': landslide_df['landslide_probability'].max(),
    'palette': ['#00ff00', '#ffff00', '#ff9900', '#ff0000'],  # Brighter colors for better visibility
    'opacity': 0.7
}

hillshade_vis = {
    'min': 0,
    'max': 255,
    'palette': ['#ffffff', '#f0f0f0', '#d0d0d0', '#a0a0a0', '#808080'],
}

# Create a GEE map
Map = geemap.Map()
Map.centerObject(boundary, 12)

# Add basemaps
Map.add_basemap('HYBRID', shown=False)  # Satellite with labels
Map.add_basemap('TERRAIN')  # Terrain base

# Add hillshade for topographic context
Map.addLayer(hillshade, hillshade_vis, 'Hillshade', True, 0.8)

# Add contour lines
Map.addLayer(contours, {'palette': '#000000'}, 'Contour Lines', True, 0.4)

# Add World Topo Map from ESRI
Map.add_basemap('Esri.WorldTopoMap', shown=True, opacity=0.6)
Map.add_basemap('OpenStreetMap', shown=False)  # OpenStreetMap

# Add the landslide susceptibility layer with transparency
Map.addLayer(landslide_susceptibility, susceptibility_vis, 'Landslide Susceptibility')

# Filter past landslide records to ensure they are within the boundary
# Convert boundary to a list of coordinates for filtering
boundary_coords = boundary.coordinates().getInfo()[0]
min_lon = min(point[0] for point in boundary_coords)
max_lon = max(point[0] for point in boundary_coords)
min_lat = min(point[1] for point in boundary_coords)
max_lat = max(point[1] for point in boundary_coords)

# Filter past_records_df to include only points within the boundary
filtered_records = past_records_df[
    (past_records_df['longitude'] >= min_lon) & 
    (past_records_df['longitude'] <= max_lon) & 
    (past_records_df['latitude'] >= min_lat) & 
    (past_records_df['latitude'] <= max_lat)
]

# Create a Folium FeatureGroup to store markers
marker_cluster = MarkerCluster(name="Past Landslide Events")

# Add past landslide records as points with better visibility
for _, row in filtered_records.iterrows():
    popup_html = f"""
    <div style="min-width:200px">
        <h4>Landslide Event</h4>
        <b>Date:</b> {row['date']}<br>
    """
    if 'magnitude' in row:
        popup_html += f"<b>Magnitude:</b> {row['magnitude']}<br>"
    if 'area' in row:
        popup_html += f"<b>Area:</b> {row['area']} m²<br>"
    popup_html += "</div>"
    
    marker = folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color='red', icon='warning-sign')  # Changed to more visible icon
    )
    marker_cluster.add_child(marker)

# Add the markers to the map
Map.add_layer(marker_cluster)

# Add a legend
legend_dict = {
    'Low Susceptibility': (0, 255, 0),      # Green
    'Moderate Susceptibility': (255, 255, 0),  # Yellow
    'High Susceptibility': (255, 165, 0),    # Orange
    'Very High Susceptibility': (255, 0, 0)   # Red
}

# Add the legend
Map.add_legend(title="Landslide Susceptibility Scale", legend_dict=legend_dict)

# Add map controls
Map.add_layer_control()


# Function to export map as high-resolution PNG
def export_map_as_png(m, output_path='landslide_susceptibility_map.png', width=3000, height=2000):
    """Export the map as a high-resolution PNG file"""
    m.to_html('temp_map.html')
    
    # Use selenium to render the map at high resolution
    try:
        from selenium import webdriver
        from selenium.webdriver.chrome.options import Options
        from selenium.webdriver.chrome.service import Service
        from webdriver_manager.chrome import ChromeDriverManager
        
        options = Options()
        options.add_argument('--headless')
        options.add_argument(f'--window-size={width},{height}')
        
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get('file://' + os.path.abspath('temp_map.html'))
        
        # Wait for map to load
        import time
        time.sleep(5)
        
        # Take screenshot
        driver.save_screenshot(output_path)
        driver.quit()
        
        print(f"Map exported as {output_path}")
        return True
    except Exception as e:
        print(f"Error exporting map: {e}")
        print("Falling back to regular screenshot method...")
        
        # Alternative: Use screenshot method (lower resolution)
        m.save('temp_map.html')
        print(f"Map saved as HTML. Please open in a browser and take a screenshot manually.")
        return False

# Add button to export high-resolution map
export_button = folium.LayerControl(position='topright')
Map.add_layer(export_button)

# Display the map
Map

# Export high-resolution map
export_map_as_png(Map, 'high_resolution_landslide_map.png')

print("To save a high-resolution version of this map, use the export_map_as_png() function.")

Error exporting map: No module named 'webdriver_manager'
Falling back to regular screenshot method...
Map saved as HTML. Please open in a browser and take a screenshot manually.
To save a high-resolution version of this map, use the export_map_as_png() function.


In [43]:
import geopandas as gpd
import pandas as pd
import numpy as np
import ee
import geemap
import folium
from folium.plugins import MarkerCluster
import os
from PIL import Image
from IPython.display import display

# Load landslide susceptibility data
landslide_df = pd.read_csv('landslide_probability.csv')
past_records_df = pd.read_csv('./map_data/final_landslide_features.csv')

# Define study area polygon
boundary = ee.Geometry.Polygon([
    [[84.39506358183932, 27.86930215221273],
     [84.39506358183932, 27.688263663637958],
     [84.57427805937839, 27.688263663637958],
     [84.57427805937839, 27.86930215221273]]
])

# Convert landslide probability data into points
landslide_points = [ee.Feature(ee.Geometry.Point([row['longitude'], row['latitude']]),
                              {'probability': float(row['landslide_probability'])})
                   for _, row in landslide_df.iterrows()]

# Create a feature collection from the points
fc = ee.FeatureCollection(landslide_points)

# Convert points to an image
landslide_image = fc.reduceToImage(
    properties=['probability'],
    reducer=ee.Reducer.mean()
)

# Apply smoothing with a focal mean filter
kernel_radius = 1000  # in meters - increased for even smoother effect
landslide_susceptibility = landslide_image.focal_mean(
    radius=kernel_radius, 
    kernelType='circle', 
    units='meters'
).clip(boundary)

# Get DEM for topographic context
dem = ee.Image('USGS/SRTMGL1_003').clip(boundary)
hillshade = ee.Terrain.hillshade(dem, 315, 45).clip(boundary)
slope = ee.Terrain.slope(dem).clip(boundary)

# Generate contour lines
contours = dem.convolve(ee.Kernel.gaussian(5)).subtract(0).divide(20).toInt()
contours = contours.mask(contours.subtract(contours.convolve(ee.Kernel.fixed(3, 3, [[1, 1, 1], [1, 0, 1], [1, 1, 1]]))))

# Create a blended landslide susceptibility and terrain visualization
# Normalize DEM for the study area for better visualization
min_elev = dem.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=boundary,
    scale=30,
    maxPixels=1e9
).get('elevation')

max_elev = dem.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=boundary,
    scale=30,
    maxPixels=1e9
).get('elevation')

dem_normalized = dem.subtract(min_elev).divide(ee.Number(max_elev).subtract(min_elev))

# Define visualization parameters with semi-transparency
susceptibility_vis = {
    'min': landslide_df['landslide_probability'].min(),
    'max': landslide_df['landslide_probability'].max(),
    'palette': ['#00ff00', '#ffff00', '#ff9900', '#ff0000'],  # Brighter colors for better visibility
    'opacity': 0.7
}

hillshade_vis = {
    'min': 0,
    'max': 255,
    'palette': ['#ffffff', '#f0f0f0', '#d0d0d0', '#a0a0a0', '#808080'],
}

# Create a GEE map
Map = geemap.Map()
Map.centerObject(boundary, 12)

# Add a light basemap to show roads without overpowering the LSM
Map.add_basemap('CartoDB.Positron')

# Add hillshade for topographic context
Map.addLayer(hillshade, hillshade_vis, 'Hillshade', True, 0.5)

# Add contour lines
Map.addLayer(contours, {'palette': '#000000'}, 'Contour Lines', True, 0.3)

# Add the landslide susceptibility layer with transparency
Map.addLayer(landslide_susceptibility, susceptibility_vis, 'Landslide Susceptibility')

# Filter past landslide records to ensure they are within the boundary
# Convert boundary to a list of coordinates for filtering
boundary_coords = boundary.coordinates().getInfo()[0]
min_lon = min(point[0] for point in boundary_coords)
max_lon = max(point[0] for point in boundary_coords)
min_lat = min(point[1] for point in boundary_coords)
max_lat = max(point[1] for point in boundary_coords)

# Filter past_records_df to include only points within the boundary
filtered_records = past_records_df[
    (past_records_df['longitude'] >= min_lon) & 
    (past_records_df['longitude'] <= max_lon) & 
    (past_records_df['latitude'] >= min_lat) & 
    (past_records_df['latitude'] <= max_lat)
]

# Create a Folium FeatureGroup to store markers
marker_cluster = MarkerCluster(name="Past Landslide Events")

# Add past landslide records as points with better visibility
for _, row in filtered_records.iterrows():
    popup_html = f"""
    <div style="min-width:200px">
        <h4>Landslide Event</h4>
        <b>Date:</b> {row['date']}<br>
    """
    if 'magnitude' in row:
        popup_html += f"<b>Magnitude:</b> {row['magnitude']}<br>"
    if 'area' in row:
        popup_html += f"<b>Area:</b> {row['area']} m²<br>"
    popup_html += "</div>"
    
    marker = folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color='red', icon='warning-sign')  # Changed to more visible icon
    )
    marker_cluster.add_child(marker)

# Add the markers to the map
Map.add_layer(marker_cluster)

# Add a legend
legend_dict = {
    'Low Susceptibility': (0, 255, 0),      # Green
    'Moderate Susceptibility': (255, 255, 0),  # Yellow
    'High Susceptibility': (255, 165, 0),    # Orange
    'Very High Susceptibility': (255, 0, 0)   # Red
}

# Add the legend
Map.add_legend(title="Landslide Susceptibility Scale", legend_dict=legend_dict)

# Display the map
Map

Map(center=[27.778786690505562, 84.48467082060839], controls=(WidgetControl(options=['position', 'transparent_…

In [5]:
import geopandas as gpd
import pandas as pd
import numpy as np
import ee
import geemap
import folium
from folium.plugins import MarkerCluster
import os
from PIL import Image
from IPython.display import display

# Load landslide susceptibility data
landslide_df = pd.read_csv('landslide_probability.csv')
past_records_df = pd.read_csv('./map_data/final_landslide_features.csv')

# Define study area polygon
boundary = ee.Geometry.Polygon([
    [[84.39506358183932, 27.86930215221273],
     [84.39506358183932, 27.688263663637958],
     [84.57427805937839, 27.688263663637958],
     [84.57427805937839, 27.86930215221273]]
])

# Convert landslide probability data into points
landslide_points = [ee.Feature(ee.Geometry.Point([row['longitude'], row['latitude']]),
                              {'probability': float(row['landslide_probability'])})
                   for _, row in landslide_df.iterrows()]

# Create a feature collection from the points
fc = ee.FeatureCollection(landslide_points)

# Convert points to an image
landslide_image = fc.reduceToImage(
    properties=['probability'],
    reducer=ee.Reducer.mean()
)

# Apply smoothing with a focal mean filter
kernel_radius = 1000  # in meters - increased for even smoother effect
landslide_susceptibility = landslide_image.focal_mean(
    radius=kernel_radius, 
    kernelType='circle', 
    units='meters'
).clip(boundary)

# Get DEM for topographic context
dem = ee.Image('USGS/SRTMGL1_003').clip(boundary)
hillshade = ee.Terrain.hillshade(dem, 315, 45).clip(boundary)
slope = ee.Terrain.slope(dem).clip(boundary)

# Generate contour lines
contours = dem.convolve(ee.Kernel.gaussian(5)).subtract(0).divide(20).toInt()
contours = contours.mask(contours.subtract(contours.convolve(ee.Kernel.fixed(3, 3, [[1, 1, 1], [1, 0, 1], [1, 1, 1]]))))

# Define visualization parameters with semi-transparency
susceptibility_vis = {
    'min': landslide_df['landslide_probability'].min(),
    'max': landslide_df['landslide_probability'].max(),
    'palette': ['#00ff00', '#ffff00', '#ff9900', '#ff0000'],  # Brighter colors for better visibility
    'opacity': 0.7
}

hillshade_vis = {
    'min': 0,
    'max': 255,
    'palette': ['#ffffff', '#f0f0f0', '#d0d0d0', '#a0a0a0', '#808080'],
}

# Create a GEE map
Map = geemap.Map()
Map.centerObject(boundary, 12)

# Add a light basemap to show roads without overpowering the LSM
Map.add_basemap('CartoDB.Positron')  # Light basemap for roads

# Add hillshade for topographic context
Map.addLayer(hillshade, hillshade_vis, 'Hillshade', True, 0.5)

# Add contour lines
Map.addLayer(contours, {'palette': '#000000'}, 'Contour Lines', True, 0.3)

# Add the landslide susceptibility layer with transparency
Map.addLayer(landslide_susceptibility, susceptibility_vis, 'Landslide Susceptibility')

# Add OpenStreetMap roads as a separate layer
# Use a custom tile layer to highlight roads
road_tile_url = 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png'
Map.add_tile_layer(road_tile_url, name='OSM Roads', attribution='OpenStreetMap', opacity=0.8)

# Filter past landslide records to ensure they are within the boundary
# Convert boundary to a list of coordinates for filtering
boundary_coords = boundary.coordinates().getInfo()[0]
min_lon = min(point[0] for point in boundary_coords)
max_lon = max(point[0] for point in boundary_coords)
min_lat = min(point[1] for point in boundary_coords)
max_lat = max(point[1] for point in boundary_coords)

# Filter past_records_df to include only points within the boundary
filtered_records = past_records_df[
    (past_records_df['longitude'] >= min_lon) & 
    (past_records_df['longitude'] <= max_lon) & 
    (past_records_df['latitude'] >= min_lat) & 
    (past_records_df['latitude'] <= max_lat)
]

# Create a Folium FeatureGroup to store markers
marker_cluster = MarkerCluster(name="Past Landslide Events")

# Add past landslide records as points with better visibility
for _, row in filtered_records.iterrows():
    popup_html = f"""
    <div style="min-width:200px">
        <h4>Landslide Event</h4>
        <b>Date:</b> {row['date']}<br>
    """
    if 'magnitude' in row:
        popup_html += f"<b>Magnitude:</b> {row['magnitude']}<br>"
    if 'area' in row:
        popup_html += f"<b>Area:</b> {row['area']} m²<br>"
    popup_html += "</div>"
    
    marker = folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color='red', icon='warning-sign')  # Changed to more visible icon
    )
    marker_cluster.add_child(marker)

# Add the markers to the map
Map.add_layer(marker_cluster)

# Add a legend
legend_dict = {
    'Low Susceptibility': (0, 255, 0),      # Green
    'Moderate Susceptibility': (255, 255, 0),  # Yellow
    'High Susceptibility': (255, 165, 0),    # Orange
    'Very High Susceptibility': (255, 0, 0)   # Red
}

# Add the legend
Map.add_legend(title="Landslide Susceptibility Scale", legend_dict=legend_dict)

# Display the map
Map

Map(center=[27.778786690505562, 84.48467082060839], controls=(WidgetControl(options=['position', 'transparent_…

In [2]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-sushantniraula01')

In [11]:
import geopandas as gpd
import pandas as pd
import numpy as np
import ee
import geemap
import folium
from folium.plugins import MarkerCluster, MiniMap
import os
from PIL import Image
from IPython.display import display

# Load landslide susceptibility data
landslide_df = pd.read_csv('landslide_probability.csv')
past_records_df = pd.read_csv('./map_data/final_landslide_features.csv')

# Define study area polygon
boundary = ee.Geometry.Polygon([
    [[84.39506358183932, 27.86930215221273],
     [84.39506358183932, 27.688263663637958],
     [84.57427805937839, 27.688263663637958],
     [84.57427805937839, 27.86930215221273]]
])

# Convert landslide probability data into points
landslide_points = [ee.Feature(ee.Geometry.Point([row['longitude'], row['latitude']]),
                              {'probability': float(row['landslide_probability'])})
                   for _, row in landslide_df.iterrows()]

# Create a feature collection from the points
fc = ee.FeatureCollection(landslide_points)

# Convert points to an image
landslide_image = fc.reduceToImage(
    properties=['probability'],
    reducer=ee.Reducer.mean()
)

# Apply smoothing with a focal mean filter
kernel_radius = 1000  # in meters - increased for even smoother effect
landslide_susceptibility = landslide_image.focal_mean(
    radius=kernel_radius, 
    kernelType='circle', 
    units='meters'
).clip(boundary)

# Get DEM for topographic context
dem = ee.Image('USGS/SRTMGL1_003').clip(boundary)
hillshade = ee.Terrain.hillshade(dem, 315, 45).clip(boundary)
slope = ee.Terrain.slope(dem).clip(boundary)

# Generate contour lines
contours = dem.convolve(ee.Kernel.gaussian(5)).subtract(0).divide(20).toInt()
contours = contours.mask(contours.subtract(contours.convolve(ee.Kernel.fixed(3, 3, [[1, 1, 1], [1, 0, 1], [1, 1, 1]]))))

# Define visualization parameters with improved ColorBrewer palette
susceptibility_vis = {
    'min': landslide_df['landslide_probability'].min(),
    'max': landslide_df['landslide_probability'].max(),
    'palette': ['#1a9850', '#ffffbf', '#fd8d3c', '#d73027'],  # ColorBrewer palette (green to red)
    'opacity': 0.75
}

hillshade_vis = {
    'min': 0,
    'max': 255,
    'palette': ['#ffffff', '#f0f0f0', '#d0d0d0', '#a0a0a0', '#808080'],
}

# Create a GEE map
Map = geemap.Map()
Map.centerObject(boundary, 12)

# Add a light basemap to show roads without overpowering the LSM
Map.add_basemap('CartoDB.Positron')  # Light basemap for roads

# Add hillshade for topographic context
Map.addLayer(hillshade, hillshade_vis, 'Hillshade', True, 0.5)

# Add contour lines
Map.addLayer(contours, {'palette': '#000000'}, 'Contour Lines', True, 0.3)

# Add the landslide susceptibility layer with transparency
Map.addLayer(landslide_susceptibility, susceptibility_vis, 'Landslide Susceptibility')

# Add OpenStreetMap roads as a separate layer
# Use a custom tile layer to highlight roads
road_tile_url = 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png'
Map.add_tile_layer(road_tile_url, name='OSM Roads', attribution='OpenStreetMap', opacity=0.8)

# --------------------------------
# Past Landslide Points Section
# --------------------------------

# Filter past landslide records to ensure they are within the boundary
# Convert boundary to a list of coordinates for filtering
boundary_coords = boundary.coordinates().getInfo()[0]
min_lon = min(point[0] for point in boundary_coords)
max_lon = max(point[0] for point in boundary_coords)
min_lat = min(point[1] for point in boundary_coords)
max_lat = max(point[1] for point in boundary_coords)

# Filter past_records_df to include only points within the boundary
filtered_records = past_records_df[
    (past_records_df['longitude'] >= min_lon) & 
    (past_records_df['longitude'] <= max_lon) & 
    (past_records_df['latitude'] >= min_lat) & 
    (past_records_df['latitude'] <= max_lat)
]

# Create a Folium FeatureGroup to store markers
marker_cluster = MarkerCluster(name="Past Landslide Events")

# Function to determine marker size based on magnitude
def get_marker_size(magnitude):
    if pd.isna(magnitude):
        return 8  # Default size
    else:
        return min(6 + magnitude * 2, 15)  # Scale size with magnitude, limit max size

# Add past landslide records as circle markers with size based on magnitude
for _, row in filtered_records.iterrows():
    popup_html = f"""
    <div style="min-width:200px">
        <h4>Landslide Event</h4>
        <b>Date:</b> {row['date']}<br>
    """
    if 'magnitude' in row:
        popup_html += f"<b>Magnitude:</b> {row['magnitude']}<br>"
    if 'area' in row:
        popup_html += f"<b>Area:</b> {row['area']} m²<br>"
    popup_html += "</div>"
    
    # Create circle marker with size based on magnitude
    marker = folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=get_marker_size(row.get('magnitude', np.nan)),
        popup=folium.Popup(popup_html, max_width=300),
        fill=True,
        fill_color='#d7301f',
        color='black',
        weight=1,
        fill_opacity=0.8,
        tooltip=f"Landslide on {row['date']}"
    )
    marker_cluster.add_child(marker)

# Add the markers to the map
Map.add_layer(marker_cluster)

# --------------------------------
# Map Enhancements Section
# --------------------------------

# Add study area boundary outline
folium_coords = [[point[1], point[0]] for point in boundary_coords]
boundary_polygon = folium.Polygon(
    locations=folium_coords,
    color='#3388ff',
    weight=3,
    fill=False,
    opacity=0.8,
    tooltip='Study Area'
)
Map.add_layer(boundary_polygon)  # Use add_layer instead of add_to



# Add custom legend for susceptibility
colors = ['#1a9850', '#ffffbf', '#fd8d3c', '#d73027']
vmin = landslide_df['landslide_probability'].min()
vmax = landslide_df['landslide_probability'].max()
labels = [f"Low: {vmin:.2f}", "Moderate", "High", f"Very High: {vmax:.2f}"]

Map.add_legend(title="Landslide Susceptibility", 
               colors=colors,
               labels=labels,
               position='bottomright')

# Add a second legend for landslide events using geemap's html method
event_legend_html = '''
<div style="
    position: fixed; 
    bottom: 120px; right: 10px; 
    border: 2px solid grey; 
    z-index: 9999; 
    background-color: white;
    padding: 5px 10px;
    border-radius: 5px;
    font-size: 12px;
    ">
    <p style="margin: 5px 0"><b>Past Landslide Events</b></p>
    <div style="display: flex; align-items: center; margin: 5px 0">
        <div style="width: 12px; height: 12px; background-color: #d7301f; border: 1px solid black; border-radius: 50%; margin-right: 5px;"></div>
        <span>Landslide Location</span>
    </div>
    <div style="margin: 5px 0; font-style: italic; font-size: 10px;">Circle size indicates magnitude</div>
</div>
'''
Map.add_html(event_legend_html, position='bottomright')

# Add layer control (geemap already has this built-in)
Map.add_layer_control()

# Display the map
Map

Map(center=[27.778786690505562, 84.48467082060839], controls=(WidgetControl(options=['position', 'transparent_…

In [22]:
import geopandas as gpd
import pandas as pd
import numpy as np
import ee
import geemap
import folium
from folium.plugins import MarkerCluster, MiniMap
import os
from PIL import Image
from IPython.display import display

ee.Authenticate()
ee.Initialize(project='ee-sushantniraula01')

# Load landslide susceptibility data
landslide_df = pd.read_csv('landslide_probability.csv')
past_records_df = pd.read_csv('./map_data/final_landslide_features.csv')

# Define study area polygon
boundary = ee.Geometry.Polygon([
    [[84.39506358183932, 27.86930215221273],
     [84.39506358183932, 27.688263663637958],
     [84.57427805937839, 27.688263663637958],
     [84.57427805937839, 27.86930215221273]]
])

# Convert landslide probability data into points
landslide_points = [ee.Feature(ee.Geometry.Point([row['longitude'], row['latitude']]),
                              {'probability': float(row['landslide_probability'])})
                   for _, row in landslide_df.iterrows()]

# Create a feature collection from the points
fc = ee.FeatureCollection(landslide_points)

# Convert points to an image
landslide_image = fc.reduceToImage(
    properties=['probability'],
    reducer=ee.Reducer.mean()
)

# Apply smoothing with a focal mean filter
kernel_radius = 1000  # in meters - increased for even smoother effect
landslide_susceptibility = landslide_image.focal_mean(
    radius=kernel_radius, 
    kernelType='circle', 
    units='meters'
).clip(boundary)

# Get DEM for topographic context
dem = ee.Image('USGS/SRTMGL1_003').clip(boundary)
hillshade = ee.Terrain.hillshade(dem, 315, 45).clip(boundary)
slope = ee.Terrain.slope(dem).clip(boundary)

# Generate contour lines
contours = dem.convolve(ee.Kernel.gaussian(5)).subtract(0).divide(20).toInt()
contours = contours.mask(contours.subtract(contours.convolve(ee.Kernel.fixed(3, 3, [[1, 1, 1], [1, 0, 1], [1, 1, 1]]))))

# Define visualization parameters with improved ColorBrewer palette
susceptibility_vis = {
    'min': landslide_df['landslide_probability'].min(),
    'max': landslide_df['landslide_probability'].max(),
    'palette': ['#1a9850', '#ffffbf', '#fd8d3c', '#d73027'],  # ColorBrewer palette (green to red)
    'opacity': 0.9
}

hillshade_vis = {
    'min': 0,
    'max': 255,
    'palette': ['#ffffff', '#f0f0f0', '#d0d0d0', '#a0a0a0', '#808080'],
}

# Create a GEE map
Map = geemap.Map()
Map.centerObject(boundary, 12)

# Add a light basemap to show roads without overpowering the LSM
Map.add_basemap('CartoDB.Positron')  # Light basemap for roads

# Add hillshade for topographic context
Map.addLayer(hillshade, hillshade_vis, 'Hillshade', True, 0.5)

# Add contour lines
Map.addLayer(contours, {'palette': '#000000'}, 'Contour Lines', True, 0.3)

# Add the landslide susceptibility layer with transparency
Map.addLayer(landslide_susceptibility, susceptibility_vis, 'Landslide Susceptibility')

# Add OpenStreetMap roads as a separate layer
# Use a custom tile layer to highlight roads
road_tile_url = 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png'
Map.add_tile_layer(road_tile_url, name='OSM Roads', attribution='OpenStreetMap', opacity=0.8)

# --------------------------------
# Past Landslide Points Section
# --------------------------------

# Filter past landslide records to ensure they are within the boundary
# Convert boundary to a list of coordinates for filtering
# Load Nepal boundary from GADM dataset
nepal_boundary = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Nepal'))
# Convert latitude and longitude to GEE Features
def create_feature(row):
    return ee.Feature(ee.Geometry.Point([row["longitude"], row["latitude"]]))

features = past_records_df.apply(create_feature, axis=1)
points_fc = ee.FeatureCollection(features.tolist())

points_inside_nepal = points_fc.filterBounds(nepal_boundary)

# Add Nepal boundary
Map.addLayer(nepal_boundary.style(color="blue", width=2), {}, "Nepal Boundary")

# Add points to the map
Map.addLayer(points_inside_nepal.style(color="black", pointSize=5), {}, "Filtered Points")



# --------------------------------
# Map Enhancements Section
# --------------------------------

# Add study area boundary outline
folium_coords = [[point[1], point[0]] for point in boundary_coords]
boundary_polygon = folium.Polygon(
    locations=folium_coords,
    color='#3388ff',
    weight=3,
    fill=False,
    opacity=0.8,
    tooltip='Study Area'
)
Map.add_layer(boundary_polygon)  # Use add_layer instead of add_to


# Add custom legend for susceptibility
colors = ['#1a9850', '#ffffbf', '#fd8d3c', '#d73027']
vmin = landslide_df['landslide_probability'].min()
vmax = landslide_df['landslide_probability'].max()
labels = [f"Low: {vmin:.2f}", "Moderate", "High", f"Very High: {vmax:.2f}"]

Map.add_legend(title="Landslide Susceptibility", 
               colors=colors,
               labels=labels,
               position='bottomright')

# Add a second legend for landslide events using geemap's html method
event_legend_html = '''
<div style="
    position: fixed; 
    bottom: 120px; right: 10px; 
    border: 2px solid grey; 
    z-index: 9999; 
    background-color: white;
    padding: 5px 10px;
    border-radius: 5px;
    font-size: 12px;
    ">
    <p style="margin: 5px 0"><b>Past Landslide Events</b></p>
    <div style="display: flex; align-items: center; margin: 5px 0">
        <div style="width: 12px; height: 12px; background-color: #d7301f; border: 1px solid black; border-radius: 50%; margin-right: 5px;"></div>
        <span>Landslide Location</span>
    </div>
    <div style="margin: 5px 0; font-style: italic; font-size: 10px;">Circle size indicates magnitude</div>
</div>
'''
Map.add_html(event_legend_html, position='bottomright')

# Add layer control (geemap already has this built-in)
Map.add_layer_control()

# Display the map
Map

Map(center=[27.778786690505562, 84.48467082060839], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
##

## Better enhanced Visuallization.


In [23]:
import geopandas as gpd
import pandas as pd
import numpy as np
import ee
import geemap
import folium
from folium.plugins import MarkerCluster, MiniMap
import os
from PIL import Image
from IPython.display import display

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-sushantniraula01')

# Load data
landslide_df = pd.read_csv('landslide_probability.csv')
past_records_df = pd.read_csv('./map_data/final_landslide_features.csv')

# Define study area polygon with more precise coordinates
boundary = ee.Geometry.Polygon([
    [[84.39506358183932, 27.86930215221273],
     [84.39506358183932, 27.688263663637958],
     [84.57427805937839, 27.688263663637958],
     [84.57427805937839, 27.86930215221273]]
])

# Convert landslide probability data to points with enhanced attributes
landslide_points = [
    ee.Feature(
        ee.Geometry.Point([row['longitude'], row['latitude']]),
        {
            'probability': float(row['landslide_probability']),
            'weight': 1.0  # Adding weight for better interpolation
        }
    ) for _, row in landslide_df.iterrows()
]

# Create feature collection
fc = ee.FeatureCollection(landslide_points)

# Convert points to image with enhanced interpolation
landslide_image = fc.reduceToImage(
    properties=['probability'],
    reducer=ee.Reducer.mean()
)

# Apply advanced smoothing with multiple passes for better visualization
kernel_radius = 1000  # meters
smoothed_image = landslide_image.focal_mean(
    radius=kernel_radius, 
    kernelType='circle', 
    units='meters'
)

# Second pass smoothing with larger kernel for edge transitions
smoothed_image = smoothed_image.focal_mean(
    radius=kernel_radius * 1.5, 
    kernelType='circle', 
    units='meters',
    iterations=2
).clip(boundary)

# Get high-resolution DEM for better topographic representation
dem = ee.Image('USGS/SRTMGL1_003').clip(boundary)

# Generate enhanced hillshade with multiple light angles for more realistic terrain
hillshade_main = ee.Terrain.hillshade(dem, 315, 45)
hillshade_secondary = ee.Terrain.hillshade(dem, 135, 60)
hillshade = hillshade_main.multiply(0.7).add(hillshade_secondary.multiply(0.3)).clip(boundary)

# Calculate slope for terrain analysis
slope = ee.Terrain.slope(dem).clip(boundary)

# Generate refined contour lines with smoother appearance
contour_interval = 20  # meters between contours
contours = dem.convolve(ee.Kernel.gaussian(5)).divide(contour_interval).floor()
contours = contours.subtract(contours.convolve(ee.Kernel.fixed(3, 3, 
                                                               [[1, 1, 1], 
                                                                [1, 0, 1], 
                                                                [1, 1, 1]])))

# Define enhanced visualization parameters with professional color palette
# Using a more nuanced color ramp similar to the reference image
susceptibility_vis = {
    'min': landslide_df['landslide_probability'].min(),
    'max': landslide_df['landslide_probability'].max(),
    'palette': [
        '#1a9850',  # Green (low risk)
        '#91cf60',  # Light green
        '#d9ef8b',  # Yellow-green
        '#ffffbf',  # Light yellow
        '#fee08b',  # Yellow
        '#fc8d59',  # Orange
        '#e34a33',  # Light red
        '#b30000'   # Deep red (high risk)
    ],
    'opacity': 0.85
}

# Enhanced hillshade visualization with greater depth
hillshade_vis = {
    'min': 0,
    'max': 255,
    'palette': ['#000000', '#1a1a1a', '#4d4d4d', '#777777', '#a6a6a6', '#d9d9d9', '#ffffff'],
    'opacity': 0.6
}

# Create map with enhanced settings
Map = geemap.Map(add_google_map=False)
Map.centerObject(boundary, 12)

# Add basemaps in specific order for better layering
Map.add_basemap('CartoDB.Positron', opacity=0.7)  # Light basemap for context

# Add hillshade as base terrain layer
Map.addLayer(hillshade, hillshade_vis, 'Hillshade', True, 0.6)

# Add contour lines with refined styling
Map.addLayer(contours, {'palette': '#555555', 'opacity': 0.4}, 'Contour Lines', True, 0.4)

# Add slope layer for additional terrain texture
Map.addLayer(slope, {'min': 0, 'max': 45, 'palette': ['#f7f7f7', '#d9d9d9', '#bdbdbd', '#969696']}, 
             'Slope', False, 0.3)

# Add the landslide susceptibility layer as semi-transparent overlay
Map.addLayer(smoothed_image, susceptibility_vis, 'Landslide Susceptibility', True, 0.8)

# Add satellite imagery for context
Map.add_basemap('HYBRID', opacity=0.3)

# Add OSM roads with refined styling
road_tile_url = 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png'
Map.add_tile_layer(road_tile_url, name='OSM Roads', attribution='OpenStreetMap', opacity=0.6)

# Process past landslide records
# Load Nepal boundary from GADM dataset
nepal_boundary = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Nepal'))

# Convert landslide points to GEE Features with size based on magnitude
def create_feature(row):
    return ee.Feature(
        ee.Geometry.Point([row["longitude"], row["latitude"]]),
        {
            'magnitude': row.get('magnitude', 1),  # Use magnitude if available
            'date': row.get('date', 'Unknown')
        }
    )

features = past_records_df.apply(create_feature, axis=1)
points_fc = ee.FeatureCollection(features.tolist())

# Filter points within boundary
points_inside_nepal = points_fc.filterBounds(nepal_boundary)

# Add Nepal boundary with subtle styling
Map.addLayer(nepal_boundary.style(color="#3388ff", width=1.5, fillColor="#3388ff00"), 
             {}, "Nepal Boundary", False)

# Add past landslide points with enhanced styling
Map.addLayer(points_inside_nepal.style(
    color="black", 
    pointSize=5,
    pointShape='circle',
    width=1,
    fillColor="#d7301f"
), {}, "Historic Landslides", True)

# Define study area boundary coordinates for folium
boundary_coords = boundary.coordinates().getInfo()[0]
folium_coords = [[point[1], point[0]] for point in boundary_coords]

# Add study area boundary with refined styling
boundary_polygon = folium.Polygon(
    locations=folium_coords,
    color='#3388ff',
    weight=2,
    fill=False,
    opacity=0.7,
    tooltip='Study Area'
)
Map.add_layer(boundary_polygon)

# Create enhanced legend for susceptibility with more detailed categories
colors = ['#1a9850', '#91cf60', '#d9ef8b', '#ffffbf', '#fee08b', '#fc8d59', '#e34a33', '#b30000']
vmin = round(landslide_df['landslide_probability'].min(), 2)
vmax = round(landslide_df['landslide_probability'].max(), 2)
mid = round((vmin + vmax) / 2, 2)
quarter = round((vmin + mid) / 2, 2)
three_quarter = round((mid + vmax) / 2, 2)

labels = [
    f"Very Low: {vmin:.2f}",
    f"Low: {quarter:.2f}",
    "Moderate-Low",
    f"Moderate: {mid:.2f}",
    "Moderate-High",
    f"High: {three_quarter:.2f}",
    "Very High",
    f"Extreme: {vmax:.2f}"
]

# Add enhanced legend with better styling
Map.add_legend(
    title="Landslide Susceptibility", 
    colors=colors,
    labels=labels,
    position='bottomright'
)

# Add historic landslide legend with improved styling
event_legend_html = '''
<div style="
    position: fixed; 
    bottom: 250px; right: 10px; 
    border: 1px solid rgba(0,0,0,0.4); 
    z-index: 9999; 
    background-color: white;
    padding: 8px 12px;
    border-radius: 4px;
    font-size: 12px;
    box-shadow: 0 1px 5px rgba(0,0,0,0.2);
    ">
    <p style="margin: 3px 0; font-weight: 600; border-bottom: 1px solid #eee; padding-bottom: 5px;">Historic Landslides</p>
    <div style="display: flex; align-items: center; margin: 5px 0">
        <div style="width: 10px; height: 10px; background-color: #d7301f; border: 1px solid #000; border-radius: 50%; margin-right: 6px;"></div>
        <span>Recorded Event</span>
    </div>
</div>
'''
Map.add_html(event_legend_html, position='bottomright')

# Add north arrow and scale bar for professional cartography
Map.add_north_arrow(position="topright", scale=0.8)
Map.add_scale_bar(position="bottomleft")

# Add layer control with categories
Map.add_layer_control()

# Set map options for better performance
Map.setOptions(
    mapTypeId='terrain',
    styles=[],
    fullscreenControl=True,
)

# Display the map
Map

AttributeError: 'Map' object has no attribute 'add_north_arrow'

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import ee
import geemap
import folium
from folium.plugins import MarkerCluster, MiniMap, MeasureControl
import os
from PIL import Image
from IPython.display import display

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-sushantniraula01')

# Load data
landslide_df = pd.read_csv('landslide_probability.csv')
past_records_df = pd.read_csv('./map_data/final_landslide_features.csv')

# Define study area polygon with more precise coordinates
boundary = ee.Geometry.Polygon([
    [[84.39506358183932, 27.86930215221273],
     [84.39506358183932, 27.688263663637958],
     [84.57427805937839, 27.688263663637958],
     [84.57427805937839, 27.86930215221273]]
])

# Convert landslide probability data to points with enhanced attributes
landslide_points = [
    ee.Feature(
        ee.Geometry.Point([row['longitude'], row['latitude']]),
        {
            'probability': float(row['landslide_probability']),
            'weight': 1.0  # Adding weight for better interpolation
        }
    ) for _, row in landslide_df.iterrows()
]

# Create feature collection
fc = ee.FeatureCollection(landslide_points)

# Convert points to image with enhanced interpolation
landslide_image = fc.reduceToImage(
    properties=['probability'],
    reducer=ee.Reducer.mean()
)

# Apply advanced smoothing with multiple passes for better visualization
kernel_radius = 1000  # meters
smoothed_image = landslide_image.focal_mean(
    radius=kernel_radius, 
    kernelType='circle', 
    units='meters'
)

# Second pass smoothing with larger kernel for edge transitions
smoothed_image = smoothed_image.focal_mean(
    radius=kernel_radius * 1.5, 
    kernelType='circle', 
    units='meters',
    iterations=2
).clip(boundary)

# Get high-resolution DEM for better topographic representation
dem = ee.Image('USGS/SRTMGL1_003').clip(boundary)

# Generate enhanced hillshade with multiple light angles for more realistic terrain
hillshade_main = ee.Terrain.hillshade(dem, 315, 45)
hillshade_secondary = ee.Terrain.hillshade(dem, 135, 60)
hillshade = hillshade_main.multiply(0.7).add(hillshade_secondary.multiply(0.3)).clip(boundary)

# Calculate slope for terrain analysis
slope = ee.Terrain.slope(dem).clip(boundary)

# Generate refined contour lines with smoother appearance
contour_interval = 20  # meters between contours
contours = dem.convolve(ee.Kernel.gaussian(5)).divide(contour_interval).floor()
contours = contours.subtract(contours.convolve(ee.Kernel.fixed(3, 3, 
                                                               [[1, 1, 1], 
                                                                [1, 0, 1], 
                                                                [1, 1, 1]])))

# Define enhanced visualization parameters with professional color palette
# Using a more nuanced color ramp similar to the reference image
susceptibility_vis = {
    'min': landslide_df['landslide_probability'].min(),
    'max': landslide_df['landslide_probability'].max(),
    'palette': [
        '#1a9850',  # Green (low risk)
        '#91cf60',  # Light green
        '#d9ef8b',  # Yellow-green
        '#ffffbf',  # Light yellow
        '#fee08b',  # Yellow
        '#fc8d59',  # Orange
        '#e34a33',  # Light red
        '#b30000'   # Deep red (high risk)
    ],
    'opacity': 0.85
}

# Enhanced hillshade visualization with greater depth
hillshade_vis = {
    'min': 0,
    'max': 255,
    'palette': ['#000000', '#1a1a1a', '#4d4d4d', '#777777', '#a6a6a6', '#d9d9d9', '#ffffff'],
    'opacity': 0.6
}

# Create map with enhanced settings
Map = geemap.Map(add_google_map=False)
Map.centerObject(boundary, 12)

# Add basemaps in specific order for better layering
Map.add_basemap('CartoDB.Positron', opacity=0.7)  # Light basemap for context

# Add hillshade as base terrain layer
Map.addLayer(hillshade, hillshade_vis, 'Hillshade', True, 0.6)

# Add contour lines with refined styling
Map.addLayer(contours, {'palette': '#555555', 'opacity': 0.4}, 'Contour Lines', True, 0.4)

# Add slope layer for additional terrain texture
Map.addLayer(slope, {'min': 0, 'max': 45, 'palette': ['#f7f7f7', '#d9d9d9', '#bdbdbd', '#969696']}, 
             'Slope', False, 0.3)

# Add the landslide susceptibility layer as semi-transparent overlay
Map.addLayer(smoothed_image, susceptibility_vis, 'Landslide Susceptibility', True, 0.8)

# Add satellite imagery for context
Map.add_basemap('HYBRID', opacity=0.3)

# Add OSM roads with refined styling
road_tile_url = 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png'
Map.add_tile_layer(road_tile_url, name='OSM Roads', attribution='OpenStreetMap', opacity=0.6)

# Process past landslide records
# Load Nepal boundary from GADM dataset
nepal_boundary = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Nepal'))

# Convert landslide points to GEE Features with size based on magnitude
def create_feature(row):
    return ee.Feature(
        ee.Geometry.Point([row["longitude"], row["latitude"]]),
        {
            'magnitude': row.get('magnitude', 1),  # Use magnitude if available
            'date': row.get('date', 'Unknown')
        }
    )

features = past_records_df.apply(create_feature, axis=1)
points_fc = ee.FeatureCollection(features.tolist())

# Filter points within boundary
points_inside_nepal = points_fc.filterBounds(nepal_boundary)

# Add Nepal boundary with subtle styling
Map.addLayer(nepal_boundary.style(color="#3388ff", width=1.5, fillColor="#3388ff00"), 
             {}, "Nepal Boundary", False)

# Add past landslide points with enhanced styling
Map.addLayer(points_inside_nepal.style(
    color="black", 
    pointSize=5,
    pointShape='circle',
    width=1,
    fillColor="#d7301f"
), {}, "Historic Landslides", True)

# Define study area boundary coordinates for folium
boundary_coords = boundary.coordinates().getInfo()[0]
folium_coords = [[point[1], point[0]] for point in boundary_coords]

# Add study area boundary with refined styling
boundary_polygon = folium.Polygon(
    locations=folium_coords,
    color='#3388ff',
    weight=2,
    fill=False,
    opacity=0.7,
    tooltip='Study Area'
)
Map.add_layer(boundary_polygon)

# Create enhanced legend for susceptibility with more detailed categories
colors = ['#1a9850', '#91cf60', '#d9ef8b', '#ffffbf', '#fee08b', '#fc8d59', '#e34a33', '#b30000']
vmin = round(landslide_df['landslide_probability'].min(), 2)
vmax = round(landslide_df['landslide_probability'].max(), 2)
mid = round((vmin + vmax) / 2, 2)
quarter = round((vmin + mid) / 2, 2)
three_quarter = round((mid + vmax) / 2, 2)

labels = [
    f"Very Low: {vmin:.2f}",
    f"Low: {quarter:.2f}",
    "Moderate-Low",
    f"Moderate: {mid:.2f}",
    "Moderate-High",
    f"High: {three_quarter:.2f}",
    "Very High",
    f"Extreme: {vmax:.2f}"
]

# Add enhanced legend with better styling
Map.add_legend(
    title="Landslide Susceptibility", 
    colors=colors,
    labels=labels,
    position='bottomright'
)

# Add historic landslide legend with improved styling
event_legend_html = '''
<div style="
    position: fixed; 
    bottom: 250px; right: 10px; 
    border: 1px solid rgba(0,0,0,0.4); 
    z-index: 9999; 
    background-color: white;
    padding: 8px 12px;
    border-radius: 4px;
    font-size: 12px;
    box-shadow: 0 1px 5px rgba(0,0,0,0.2);
    ">
    <p style="margin: 3px 0; font-weight: 600; border-bottom: 1px solid #eee; padding-bottom: 5px;">Historic Landslides</p>
    <div style="display: flex; align-items: center; margin: 5px 0">
        <div style="width: 10px; height: 10px; background-color: #d7301f; border: 1px solid #000; border-radius: 50%; margin-right: 6px;"></div>
        <span>Recorded Event</span>
    </div>
</div>
'''
Map.add_html(event_legend_html, position='bottomright')

# Add a manual north arrow using custom HTML
north_arrow_html = '''
<div style="
    position: fixed; 
    top: 10px; right: 10px; 
    z-index: 9999; 
    background-color: rgba(255, 255, 255, 0.8);
    padding: 5px;
    border-radius: 4px;
    box-shadow: 0 1px 5px rgba(0,0,0,0.2);
    width: 30px;
    height: 30px;
    text-align: center;
    ">
    <div style="font-weight: bold; font-size: 18px;">N</div>
    <div style="position: absolute; top: 3px; left: 14px; width: 1px; height: 15px; background-color: black; transform: rotate(0deg);"></div>
    <div style="position: absolute; top: 17px; left: 10px; width: 10px; height: 1px; background-color: black; transform: rotate(0deg);"></div>
    <div style="position: absolute; top: 17px; left: 14px; width: 10px; height: 1px; background-color: black; transform: rotate(90deg);"></div>
</div>
'''
Map.add_html(north_arrow_html)

# Add a manual scale bar (since the native add_scale_bar might not be available)
# Folium has a built-in scale control, let's use that instead
Map.to_streamlit = lambda: Map

# Add MeasureControl from folium.plugins
measure = MeasureControl(
    position='bottomleft',
    primary_length_unit='kilometers',
    secondary_length_unit='miles',
    primary_area_unit='square kilometers',
    secondary_area_unit='acres'
)
Map.add(measure)

# Add MiniMap for context
minimap = MiniMap(
    position='bottomleft',
    toggle_display=True,
    tile_layer='CartoDB.Positron'
)
Map.add(minimap)

# Add layer control with categories
Map.add_layer_control()

# Set map options for better performance
Map.setOptions(
    mapTypeId='terrain',
    styles=[],
    fullscreenControl=True,
)

# Display the map
Map

Map(center=[27.778786690505562, 84.48467082060839], controls=(WidgetControl(options=['position', 'transparent_…

In [26]:
import geopandas as gpd
import pandas as pd
import numpy as np
import ee
import geemap
import folium
from folium.plugins import MarkerCluster, MiniMap, MeasureControl
import os
from PIL import Image
from IPython.display import display

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-sushantniraula01')

# Load data
landslide_df = pd.read_csv('landslide_probability.csv')
past_records_df = pd.read_csv('./map_data/final_landslide_features.csv')

# Define study area polygon with more precise coordinates
boundary = ee.Geometry.Polygon([
    [[84.39506358183932, 27.86930215221273],
     [84.39506358183932, 27.688263663637958],
     [84.57427805937839, 27.688263663637958],
     [84.57427805937839, 27.86930215221273]]
])

# Convert landslide probability data to points with enhanced attributes
landslide_points = [
    ee.Feature(
        ee.Geometry.Point([row['longitude'], row['latitude']]),
        {
            'probability': float(row['landslide_probability']),
            'weight': 1.0  # Adding weight for better interpolation
        }
    ) for _, row in landslide_df.iterrows()
]

# Create feature collection
fc = ee.FeatureCollection(landslide_points)

# Convert points to image with enhanced interpolation
landslide_image = fc.reduceToImage(
    properties=['probability'],
    reducer=ee.Reducer.mean()
)

# Apply advanced smoothing with multiple passes for better visualization
kernel_radius = 1000  # meters
smoothed_image = landslide_image.focal_mean(
    radius=kernel_radius, 
    kernelType='circle', 
    units='meters'
)

# Second pass smoothing with larger kernel for edge transitions
smoothed_image = smoothed_image.focal_mean(
    radius=kernel_radius * 1.5, 
    kernelType='circle', 
    units='meters',
    iterations=2
).clip(boundary)

# Define enhanced visualization parameters with professional color palette
susceptibility_vis = {
    'min': landslide_df['landslide_probability'].min(),
    'max': landslide_df['landslide_probability'].max(),
    'palette': [
        '#1a9850',  # Green (low risk)
        '#91cf60',  # Light green
        '#d9ef8b',  # Yellow-green
        '#ffffbf',  # Light yellow
        '#fee08b',  # Yellow
        '#fc8d59',  # Orange
        '#e34a33',  # Light red
        '#b30000'   # Deep red (high risk)
    ],
    'opacity': 0.85
}

# Create map with enhanced settings
Map = geemap.Map(add_google_map=False)
Map.centerObject(boundary, 12)

# Add basemaps in specific order for better layering
Map.add_basemap('CartoDB.Positron', opacity=0.7)  # Light basemap for context

# Add the landslide susceptibility layer as semi-transparent overlay
Map.addLayer(smoothed_image, susceptibility_vis, 'Landslide Susceptibility', True, 0.8)

# Add satellite imagery for context
Map.add_basemap('HYBRID', opacity=0.3)

# Add OSM roads with refined styling
road_tile_url = 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png'
Map.add_tile_layer(road_tile_url, name='OSM Roads', attribution='OpenStreetMap', opacity=0.6)

# Create enhanced legend for susceptibility with more detailed categories
colors = ['#1a9850', '#91cf60', '#d9ef8b', '#ffffbf', '#fee08b', '#fc8d59', '#e34a33', '#b30000']
vmin = round(landslide_df['landslide_probability'].min(), 2)
vmax = round(landslide_df['landslide_probability'].max(), 2)
mid = round((vmin + vmax) / 2, 2)
quarter = round((vmin + mid) / 2, 2)
three_quarter = round((mid + vmax) / 2, 2)

labels = [
    f"Very Low: {vmin:.2f}",
    f"Low: {quarter:.2f}",
    "Moderate-Low",
    f"Moderate: {mid:.2f}",
    "Moderate-High",
    f"High: {three_quarter:.2f}",
    "Very High",
    f"Extreme: {vmax:.2f}"
]

# Add enhanced legend with better styling
Map.add_legend(
    title="Landslide Susceptibility", 
    colors=colors,
    labels=labels,
    position='bottomright'
)

# Add a manual north arrow using custom HTML
north_arrow_html = '''
<div style="
    position: fixed; 
    top: 10px; right: 10px; 
    z-index: 9999; 
    background-color: rgba(255, 255, 255, 0.8);
    padding: 5px;
    border-radius: 4px;
    box-shadow: 0 1px 5px rgba(0,0,0,0.2);
    width: 30px;
    height: 30px;
    text-align: center;
    ">
    <div style="font-weight: bold; font-size: 18px;">N</div>
    <div style="position: absolute; top: 3px; left: 14px; width: 1px; height: 15px; background-color: black; transform: rotate(0deg);"></div>
    <div style="position: absolute; top: 17px; left: 10px; width: 10px; height: 1px; background-color: black; transform: rotate(0deg);"></div>
    <div style="position: absolute; top: 17px; left: 14px; width: 10px; height: 1px; background-color: black; transform: rotate(90deg);"></div>
</div>
'''
Map.add_html(north_arrow_html)

# Add MeasureControl from folium.plugins
measure = MeasureControl(
    position='bottomleft',
    primary_length_unit='kilometers',
    secondary_length_unit='miles',
    primary_area_unit='square kilometers',
    secondary_area_unit='acres'
)
Map.add(measure)

# Add MiniMap for context
minimap = MiniMap(
    position='bottomleft',
    toggle_display=True,
    tile_layer='CartoDB.Positron'
)
Map.add(minimap)

# Add layer control with categories
Map.add_layer_control()

# Set map options for better performance
Map.setOptions(
    mapTypeId='terrain',
    styles=[],
    fullscreenControl=True,
)

# Display the map
Map

Map(center=[27.778786690505562, 84.48467082060839], controls=(WidgetControl(options=['position', 'transparent_…